In [15]:
import dash
from dash import dcc, html, Input, Output, State, dash_table, ctx
import dash_bootstrap_components as dbc
import mysql.connector
import base64
import os
import datetime
import uuid
import flask
from mysql.connector import Error
import pandas as pd

# Add this at the top with your other imports
from flask import send_from_directory


# Make sure to add a reference to the app.server
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
server = app.server  # Important: Make Flask server accessible

# Configuration
IMAGE_UPLOAD_DIRECTORY = '/Users/manouellahelou/Desktop/Project/images'
DB_CONFIG = {
    'host': 'localhost',
    'user': 'root',
    'password': 'Mysql@1234', 
    'database': 'iq_test_db'
}

# Ensure the upload directory exists
os.makedirs(IMAGE_UPLOAD_DIRECTORY, exist_ok=True)

# Initialize the Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.title = "IQ Test Admin Dashboard"

# Database connection function
def get_db_connection():
    try:
        connection = mysql.connector.connect(**DB_CONFIG)
        return connection
    except Error as e:
        print(f"Error connecting to MySQL database: {e}")
        return None

# Function to get categories from the database
def get_categories():
    connection = get_db_connection()
    if connection:
        cursor = connection.cursor(dictionary=True)
        try:
            cursor.execute("SELECT * FROM categories")
            categories = cursor.fetchall()
            return categories
        except Error as e:
            print(f"Error fetching categories: {e}")
            return []
        finally:
            cursor.close()
            connection.close()
    return []

# Function to get questions from the database
def get_questions():
    connection = get_db_connection()
    if connection:
        cursor = connection.cursor(dictionary=True)
        try:
            query = """
            SELECT q.question_id, q.question_text, q.image_path, q.difficulty, 
                   q.section_id, q.num_options, q.question_type,
                   c.name as category_name, c.category_id
            FROM questions q
            JOIN categories c ON q.category_id = c.category_id
            ORDER BY q.question_id DESC
            """
            cursor.execute(query)
            questions = cursor.fetchall()
            return questions
        except Error as e:
            print(f"Error fetching questions: {e}")
            return []
        finally:
            cursor.close()
            connection.close()
    return []

# Function to get answers for a specific question
def get_answers(question_id):
    connection = get_db_connection()
    if connection:
        cursor = connection.cursor(dictionary=True)
        try:
            cursor.execute("SELECT * FROM answer_options WHERE question_id = %s", (question_id,))
            answers = cursor.fetchall()
            return answers
        except Error as e:
            print(f"Error fetching answers: {e}")
            return []
        finally:
            cursor.close()
            connection.close()
    return []

# Function to delete a question and its answers
def delete_question(question_id):
    connection = get_db_connection()
    if connection:
        cursor = connection.cursor()
        try:
            # First get the image path to delete the file if it exists
            cursor.execute("SELECT image_path FROM questions WHERE question_id = %s", (question_id,))
            result = cursor.fetchone()
            
            if result and result[0]:
                image_path = os.path.join(IMAGE_UPLOAD_DIRECTORY, result[0])
                if os.path.exists(image_path):
                    os.remove(image_path)
            
            # Delete answers
            cursor.execute("DELETE FROM answer_options WHERE question_id = %s", (question_id,))
            
            # Delete question
            cursor.execute("DELETE FROM questions WHERE question_id = %s", (question_id,))
            
            connection.commit()
            return True
        except Error as e:
            print(f"Error deleting question: {e}")
            connection.rollback()
            return False
        finally:
            cursor.close()
            connection.close()
    return False

# Create initial categories if they don't exist
def initialize_categories():
    connection = get_db_connection()
    if connection:
        cursor = connection.cursor()
        try:
            # Check if categories exist
            cursor.execute("SELECT COUNT(*) FROM categories")
            count = cursor.fetchone()[0]
            
            # If no categories, create default ones
            if count == 0:
                default_categories = [
                    ("Logical Reasoning", "Questions that test logical thinking ability"),
                    ("Spatial Awareness", "Questions that test spatial visualization skills"),
                    ("Pattern Recognition", "Questions that test ability to identify patterns"),
                    ("Mathematical Reasoning", "Questions that test mathematical problem-solving"),
                    ("Verbal Reasoning", "Questions that test language comprehension"),
                    ("Non-verbal Reasoning", "Questions that test reasoning with visuals and patterns"),
                    ("Business Understanding", "Questions that test business concepts and scenarios")
                ]
                
                for name, description in default_categories:
                    cursor.execute(
                        "INSERT INTO categories (name, description) VALUES (%s, %s)",
                        (name, description)
                    )
                
                connection.commit()
                print("Default categories created successfully")
        except Error as e:
            print(f"Error initializing categories: {e}")
        finally:
            cursor.close()
            connection.close()

# Initialize categories when the app starts
initialize_categories()

# Layout for the Dash app
app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.H1("IQ Test Question Admin Dashboard", className="text-center my-4"),
            html.Hr()
        ])
    ]),
    
    # Tabs for different functions
    dbc.Tabs(
        [
            # Question Entry Tab
            dbc.Tab(
                label="Add Questions", 
                tab_id="tab-questions",
                children=[
                    dbc.Row([
                        dbc.Col([
                            dbc.Card([
                                dbc.CardHeader(html.H3("Add New Question")),
                                dbc.CardBody([
                                    # Section selection
                                    dbc.Row([
                                        dbc.Col([
                                            dbc.Label("Section:"),
                                            dcc.Dropdown(
                                                id="section-dropdown",
                                                options=[
                                                    {"label": "Section 1 (Non-Verbal Reasoning)", "value": 1},
                                                    {"label": "Section 2 (Logical Reasoning)", "value": 2},
                                                    {"label": "Section 3 (Business Understanding)", "value": 3}
                                                ],
                                                value=1
                                            )
                                        ], width=12, className="mb-3")
                                    ]),
                                    
                                    # Category selection - will be updated based on section
                                    dbc.Row([
                                        dbc.Col([
                                            dbc.Label("Category:"),
                                            dcc.Dropdown(
                                                id="category-dropdown",
                                                options=[
                                                    {"label": cat["name"], "value": cat["category_id"]} 
                                                    for cat in get_categories()
                                                ]
                                            )
                                        ], width=12, className="mb-3")
                                    ]),
                                    
                                    # Question type selection
                                    dbc.Row([
                                        dbc.Col([
                                            dbc.Label("Question Type:"),
                                            dcc.Dropdown(
                                                id="question-type-dropdown",
                                                options=[
                                                    {"label": "Text Only", "value": "text_only"},
                                                    {"label": "Image Only", "value": "image_only"},
                                                    {"label": "Text and Image", "value": "text_and_image"}
                                                ],
                                                value="text_only"
                                            )
                                        ], width=12, className="mb-3")
                                    ]),
                                    
                                    # Text input (conditional)
                                    html.Div(
                                        dbc.Row([
                                            dbc.Col([
                                                dbc.Label("Question Text:"),
                                                dbc.Textarea(
                                                    id="question-text-input",
                                                    placeholder="Enter the question text here...",
                                                    style={"height": "150px"}
                                                )
                                            ], width=12)
                                        ], className="mb-3"),
                                        id="text-input-container"
                                    ),
                                    
                                    # Image upload (conditional)
                                    html.Div(
                                        dbc.Row([
                                            dbc.Col([
                                                dbc.Label("Question Image:"),
                                                dcc.Upload(
                                                    id="question-image-upload",
                                                    children=html.Div([
                                                        'Drag and Drop or ',
                                                        html.A('Select an Image')
                                                    ]),
                                                    style={
                                                        'width': '100%',
                                                        'height': '60px',
                                                        'lineHeight': '60px',
                                                        'borderWidth': '1px',
                                                        'borderStyle': 'dashed',
                                                        'borderRadius': '5px',
                                                        'textAlign': 'center',
                                                        'margin': '10px'
                                                    },
                                                    multiple=False
                                                ),
                                                html.Div(id="image-preview")
                                            ], width=12)
                                        ], className="mb-3"),
                                        id="image-upload-container",
                                        style={"display": "none"}
                                    ),
                                    
                                    # Difficulty selection
                                    dbc.Row([
                                        dbc.Col([
                                            dbc.Label("Difficulty (1-10):"),
                                            dcc.Slider(
                                                id="difficulty-slider",
                                                min=1,
                                                max=10,
                                                step=1,
                                                marks={i: str(i) for i in range(1, 11)},
                                                value=5
                                            )
                                        ], width=12, className="mb-3")
                                    ]),
                                    
                                    # Number of options selection
                                    dbc.Row([
                                        dbc.Col([
                                            dbc.Label("Number of Options:"),
                                            dcc.RadioItems(
                                                id="num-options-radio",
                                                options=[
                                                    {"label": "4 Options (A-D)", "value": 4},
                                                    {"label": "6 Options (A-F)", "value": 6}
                                                ],
                                                value=4,
                                                labelStyle={"display": "block", "margin-bottom": "5px"}
                                            )
                                        ], width=12, className="mb-3")
                                    ]),
                                    
                                    # Options input fields (dynamically generated)
                                    html.Div(id="options-container"),
                                    
                                    # Submit button
                                    dbc.Button(
                                        "Add Question", 
                                        id="submit-button", 
                                        color="primary", 
                                        className="mt-3"
                                    ),
                                    
                                    # Feedback alert
                                    html.Div(id="submit-feedback")
                                ])
                            ])
                        ], width=8, className="mx-auto")
                    ])
                ]
            ),
            
            # Category Management Tab
            dbc.Tab(
                label="Manage Categories", 
                tab_id="tab-categories",
                children=[
                    dbc.Row([
                        dbc.Col([
                            dbc.Card([
                                dbc.CardHeader(html.H3("Category Management")),
                                dbc.CardBody([
                                    # Add new category form
                                    dbc.Row([
                                        dbc.Col([
                                            html.H4("Add New Category"),
                                            dbc.Row([
                                                dbc.Col([
                                                    dbc.Label("Category Name:"),
                                                    dbc.Input(
                                                        id="category-name-input",
                                                        type="text",
                                                        placeholder="Enter category name"
                                                    )
                                                ], width=12, className="mb-3"),
                                            ]),
                                            dbc.Row([
                                                dbc.Col([
                                                    dbc.Label("Description:"),
                                                    dbc.Textarea(
                                                        id="category-description-input",
                                                        placeholder="Enter category description (optional)",
                                                        style={"height": "100px"}
                                                    )
                                                ], width=12, className="mb-3"),
                                            ]),
                                            dbc.Button(
                                                "Add Category", 
                                                id="add-category-button", 
                                                color="success", 
                                                className="mb-3"
                                            ),
                                            html.Div(id="category-feedback"),
                                        ], width=12)
                                    ]),
                                    
                                    html.Hr(),
                                    
                                    # List of existing categories
                                    dbc.Row([
                                        dbc.Col([
                                            html.H4("Existing Categories"),
                                            html.Div(id="categories-table"),
                                        ], width=12)
                                    ])
                                ])
                            ])
                        ], width=8, className="mx-auto")
                    ])
                ]
            ),
            
            # New Tab: View/Manage Questions
            dbc.Tab(
                label="View Questions", 
                tab_id="tab-view-questions",
                children=[
                    dbc.Row([
                        dbc.Col([
                            dbc.Card([
                                dbc.CardHeader(html.H3("View and Manage Questions")),
                                dbc.CardBody([
                                    # Filter options
                                    dbc.Row([
                                        dbc.Col([
                                            dbc.Label("Filter by Section:"),
                                            dcc.Dropdown(
                                                id="filter-section-dropdown",
                                                options=[
                                                    {"label": "All Sections", "value": "all"},
                                                    {"label": "Section 1 (Non-Verbal Reasoning)", "value": 1},
                                                    {"label": "Section 2 (Logical Reasoning)", "value": 2},
                                                    {"label": "Section 3 (Business Understanding)", "value": 3}
                                                ],
                                                value="all"
                                            )
                                        ], width=6),
                                        
                                        dbc.Col([
                                            dbc.Label("Filter by Category:"),
                                            dcc.Dropdown(
                                                id="filter-category-dropdown",
                                                options=[
                                                    {"label": "All Categories", "value": "all"},
                                                ] + [
                                                    {"label": cat["name"], "value": cat["category_id"]} 
                                                    for cat in get_categories()
                                                ],
                                                value="all"
                                            )
                                        ], width=6),
                                    ], className="mb-3"),
                                    
                                    # Question list
                                    dbc.Row([
                                        dbc.Col([
                                            html.Div(id="questions-table"),
                                        ], width=12)
                                    ]),
                                    
                                    # Question detail modal
                                    dbc.Modal(
                                        [
                                            dbc.ModalHeader(dbc.ModalTitle("Question Details")),
                                            dbc.ModalBody(id="question-detail-body"),
                                            dbc.ModalFooter(
                                                dbc.Button(
                                                    "Close", id="close-question-modal", className="ms-auto", n_clicks=0
                                                )
                                            ),
                                        ],
                                        id="question-detail-modal",
                                        size="lg",
                                        is_open=False,
                                    ),
                                    
                                    # Confirmation modal for deletion
                                    dbc.Modal(
                                        [
                                            dbc.ModalHeader(dbc.ModalTitle("Confirm Deletion")),
                                            dbc.ModalBody("Are you sure you want to delete this question? This action cannot be undone."),
                                            dbc.ModalFooter([
                                                dbc.Button(
                                                    "Cancel", id="cancel-delete", className="me-1", n_clicks=0
                                                ),
                                                dbc.Button(
                                                    "Delete", id="confirm-delete", color="danger", n_clicks=0
                                                ),
                                            ]),
                                        ],
                                        id="delete-confirmation-modal",
                                        is_open=False,
                                    ),
                                    
                                    # Store the question ID to delete
                                    dcc.Store(id="question-to-delete"),
                                    
                                    # Feedback for delete operation
                                    html.Div(id="delete-feedback", className="mt-3")
                                ])
                            ])
                        ], width=10, className="mx-auto")
                    ])
                ]
            )
        ],
        id="main-tabs",
        active_tab="tab-questions"
    )
], fluid=True)

# Callback to update categories dropdown when tab is clicked or section is changed
@app.callback(
    Output("category-dropdown", "options"),
    Output("category-dropdown", "value"),
    [Input("main-tabs", "active_tab"),
     Input("section-dropdown", "value")]
)
def update_category_dropdown(active_tab, section_id):
    categories = get_categories()
    
    # Map section IDs to default category names
    section_to_category = {
        1: "Non-verbal Reasoning",
        2: "Logical Reasoning",
        3: "Business Understanding"
    }
    
    # Find the category ID for the default category
    default_category_id = None
    if section_id in section_to_category:
        default_category_name = section_to_category[section_id]
        for cat in categories:
            if cat["name"] == default_category_name:
                default_category_id = cat["category_id"]
                break
    
    # If no default category found, use the first category (if any)
    if default_category_id is None and categories:
        default_category_id = categories[0]["category_id"]
    
    return [{"label": cat["name"], "value": cat["category_id"]} for cat in categories], default_category_id

# Callback to show/hide text and image inputs based on question type
@app.callback(
    [Output("text-input-container", "style"),
     Output("image-upload-container", "style")],
    [Input("question-type-dropdown", "value")]
)
def update_input_visibility(question_type):
    text_style = {"display": "block"} if question_type in ["text_only", "text_and_image"] else {"display": "none"}
    image_style = {"display": "block"} if question_type in ["image_only", "text_and_image"] else {"display": "none"}
    return text_style, image_style

# Callback to generate option inputs based on number of options
@app.callback(
    Output("options-container", "children"),
    [Input("num-options-radio", "value")]
)
def generate_option_inputs(num_options):
    options = []
    options_labels = [chr(65 + i) for i in range(num_options)]  # A, B, C, D, (E, F)
    
    for i, label in enumerate(options_labels):
        option_group = dbc.Row([
            dbc.Col([
                dbc.Label(f"Option {label}:"),
                dbc.Row([
                    dbc.Col([
                        dbc.Input(
                            id={"type": "option-input", "index": i},
                            placeholder=f"Enter option {label}",
                            type="text"
                        ),
                    ], width=10),
                    dbc.Col([
                        html.Div([
                            dcc.Checklist(
                                id={"type": "option-correct", "index": i},
                                options=[{"label": "Correct", "value": "correct"}],
                                value=[],
                                inline=True
                            )
                        ])
                    ], width=2)
                ])
            ], width=12)
        ], className="mb-2")
        options.append(option_group)
    
    return options

# Callback to handle image upload and show preview
@app.callback(
    Output("image-preview", "children"),
    [Input("question-image-upload", "contents")],
    [State("question-image-upload", "filename")]
)
def update_image_preview(contents, filename):
    if contents is None:
        return []
    
    # Display the image preview
    return [
        html.Div([
            html.Hr(),
            html.P(f"Selected file: {filename}"),
            html.Img(src=contents, style={"max-width": "100%", "max-height": "300px"})
        ])
    ]

# Function to save the uploaded image
def save_uploaded_image(contents, filename):
    if contents is None:
        return None
    
    try:
        # Generate a unique filename to avoid collisions
        file_extension = os.path.splitext(filename)[1].lower()
        unique_filename = f"{str(uuid.uuid4())}{file_extension}"
        file_path = os.path.join(IMAGE_UPLOAD_DIRECTORY, unique_filename)
        
        # Decode and save the image
        content_type, content_string = contents.split(',')
        decoded = base64.b64decode(content_string)
        
        with open(file_path, 'wb') as f:
            f.write(decoded)
        
        return unique_filename
    except Exception as e:
        print(f"Error saving image: {e}")
        return None

# Callback to add a new category
@app.callback(
    [Output("category-feedback", "children"),
     Output("category-name-input", "value"),
     Output("category-description-input", "value"),
     Output("categories-table", "children")],
    [Input("add-category-button", "n_clicks")],
    [State("category-name-input", "value"),
     State("category-description-input", "value")]
)
def add_category(n_clicks, name, description):
    if n_clicks is None or n_clicks == 0:
        # Initial load
        return None, "", "", create_categories_table()
    
    if not name:
        return dbc.Alert("Please enter a category name", color="danger"), name, description, create_categories_table()
    
    connection = get_db_connection()
    if connection:
        cursor = connection.cursor()
        try:
            # Check if category already exists
            cursor.execute("SELECT COUNT(*) FROM categories WHERE name = %s", (name,))
            count = cursor.fetchone()[0]
            
            if count > 0:
                return dbc.Alert(f"Category '{name}' already exists", color="warning"), name, description, create_categories_table()
            
            # Insert the new category
            cursor.execute(
                "INSERT INTO categories (name, description) VALUES (%s, %s)",
                (name, description or "")
            )
            connection.commit()
            
            return dbc.Alert(f"Category '{name}' added successfully", color="success"), "", "", create_categories_table()
            
        except Error as e:
            print(f"Error adding category: {e}")
            return dbc.Alert(f"Error: {str(e)}", color="danger"), name, description, create_categories_table()
        finally:
            cursor.close()
            connection.close()
    
    return dbc.Alert("Database connection failed", color="danger"), name, description, create_categories_table()

# Function to create a table of categories
def create_categories_table():
    categories = get_categories()
    
    if not categories:
        return html.P("No categories found. Add your first category above.")
    
    # Create table headers
    table_header = html.Thead(html.Tr([
        html.Th("ID"), 
        html.Th("Name"), 
        html.Th("Description")
    ]))
    
    # Create table rows
    rows = []
    for category in categories:
        row = html.Tr([
            html.Td(category["category_id"]),
            html.Td(category["name"]),
            html.Td(category["description"] or "")
        ])
        rows.append(row)
    
    table_body = html.Tbody(rows)
    
    # Create the table
    table = dbc.Table(
        [table_header, table_body],
        bordered=True,
        hover=True,
        striped=True,
        responsive=True
    )
    
    return table

# Callback to handle question submission
@app.callback(
    Output("submit-feedback", "children"),
    [Input("submit-button", "n_clicks")],
    [State("section-dropdown", "value"),
     State("category-dropdown", "value"),
     State("question-type-dropdown", "value"),
     State("question-text-input", "value"),
     State("question-image-upload", "contents"),
     State("question-image-upload", "filename"),
     State("difficulty-slider", "value"),
     State("num-options-radio", "value"),
     State({"type": "option-input", "index": dash.ALL}, "value"),
     State({"type": "option-correct", "index": dash.ALL}, "value")]
)
def submit_question(n_clicks, section_id, category_id, question_type, question_text, 
                   image_contents, image_filename, difficulty, num_options,
                   option_values, option_correct_values):
    if n_clicks is None or n_clicks == 0:
        return None
    
    # Validate inputs
    if not category_id:
        return dbc.Alert("Please select a category", color="danger")
    
    if question_type in ["text_only", "text_and_image"] and not question_text:
        return dbc.Alert("Please enter question text", color="danger")
    
    if question_type in ["image_only", "text_and_image"] and not image_contents:
        return dbc.Alert("Please upload an image", color="danger")
    
    # Convert checklist values to boolean
    option_correct = [len(val) > 0 for val in option_correct_values]
    
    # Check if at least one correct answer is selected
    if not any(option_correct[:num_options]):
        return dbc.Alert("Please select at least one correct answer", color="danger")
    
    # Check if all options have text
    for i in range(num_options):
        if not option_values[i]:
            return dbc.Alert(f"Please enter text for Option {chr(65+i)}", color="danger")
    
    try:
        # Save the image if applicable
        image_path = None
        if question_type in ["image_only", "text_and_image"]:
            image_path = save_uploaded_image(image_contents, image_filename)
        
        # Connect to the database
        connection = get_db_connection()
        if not connection:
            return dbc.Alert("Database connection failed", color="danger")
        
        cursor = connection.cursor()
        
        # Insert the question
        query = """
        INSERT INTO questions 
        (category_id, section_id, question_type, question_text, image_path, 
         difficulty, num_options, created_by, created_at)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        
        # Assuming user_id 1 for admin
        user_id = 1
        current_time = datetime.datetime.now()
        
        cursor.execute(query, (
            category_id, section_id, question_type, question_text, image_path,
            difficulty, num_options, user_id, current_time
        ))
        
        # Get the inserted question ID
        question_id = cursor.lastrowid
        
        # Insert the options
        for i in range(num_options):
            option_text = option_values[i]
            is_correct = len(option_correct_values[i]) > 0  # Convert checklist value to boolean
            
            cursor.execute(
                "INSERT INTO answer_options (question_id, option_text, is_correct) VALUES (%s, %s, %s)",
                (question_id, option_text, is_correct)
            )
        
        connection.commit()
        
        # Success message
        return dbc.Alert(
            f"Question added successfully! (ID: {question_id})", 
            color="success"
        )
        
    except Error as e:
        print(f"Error adding question: {e}")
        return dbc.Alert(f"Error: {str(e)}", color="danger")
    
    finally:
        if 'connection' in locals() and connection.is_connected():
            cursor.close()
            connection.close()

# Callbacks for the View Questions tab
@app.callback(
    Output("questions-table", "children"),
    [Input("main-tabs", "active_tab"),
     Input("filter-section-dropdown", "value"),
     Input("filter-category-dropdown", "value"),
     Input("delete-feedback", "children")]  # Trigger refresh when a question is deleted
)
def update_questions_table(active_tab, section_filter, category_filter, _):
    if active_tab != "tab-view-questions":
        return []
    
    questions = get_questions()
    
    if not questions:
        return html.P("No questions found in the database.")
    
    # Apply filters
    filtered_questions = questions
    if section_filter != "all":
        filtered_questions = [q for q in filtered_questions if q["section_id"] == int(section_filter)]
    if category_filter != "all":
        filtered_questions = [q for q in filtered_questions if q["category_id"] == int(category_filter)]
    
    if not filtered_questions:
        return html.P("No questions match the selected filters.")
    
    # Create table headers
    table_header = html.Thead(html.Tr([
        html.Th("ID"),
        html.Th("Question"),
        html.Th("Section"),
        html.Th("Category"),
        html.Th("Difficulty"),
        html.Th("Type"),
        html.Th("Actions")
    ]))
    
    # Create table rows
    rows = []
    for question in filtered_questions:
        # Format question text for table (truncate if needed)
        q_text = question.get("question_text", "")
        if q_text and len(q_text) > 50:
            q_text = q_text[:50] + "..."
        elif not q_text and question.get("image_path"):
            q_text = "[Image Only]"
        
        # Get section name
        section_name = {
            1: "Section 1 (Non-Verbal Reasoning)",
            2: "Section 2 (Logical Reasoning)",
            3: "Section 3 (Business Understanding)"
        }.get(question["section_id"], f"Section {question['section_id']}")
        
        # Create action buttons
        view_button = dbc.Button(
            "View", 
            id={"type": "view-question", "index": question["question_id"]},
            color="info", 
            size="sm",
            className="me-1"
        )
        
        delete_button = dbc.Button(
            "Delete", 
            id={"type": "delete-question", "index": question["question_id"]},
            color="danger", 
            size="sm"
        )
        
        row = html.Tr([
            html.Td(question["question_id"]),
            html.Td(q_text),
            html.Td(section_name),
            html.Td(question["category_name"]),
            html.Td(question["difficulty"]),
            html.Td(question["question_type"].replace("_", " ").title()),
            html.Td([view_button, delete_button])
        ])
        rows.append(row)
    
    table_body = html.Tbody(rows)
    
    # Create the table
    table = dbc.Table(
        [table_header, table_body],
        bordered=True,
        hover=True,
        striped=True,
        responsive=True
    )
    
    return table

# Callback to open question detail modal
@app.callback(
    [Output("question-detail-modal", "is_open"),
     Output("question-detail-body", "children")],
    [Input({"type": "view-question", "index": dash.ALL}, "n_clicks")],
    [State("question-detail-modal", "is_open")]
)
def toggle_question_detail_modal(n_clicks_list, is_open):
    if not n_clicks_list or not any(n_clicks for n_clicks in n_clicks_list if n_clicks):
        return is_open, []
    
    # Find which button was clicked
    triggered_id = ctx.triggered_id
    if triggered_id is None:
        return is_open, []
    
    question_id = triggered_id["index"]
    
    # Get question details
    questions = get_questions()
    question = next((q for q in questions if q["question_id"] == question_id), None)
    
    if not question:
        return is_open, [html.P("Question not found.")]
    
    # Get answers for this question
    answers = get_answers(question_id)
    
    # Create detail view
    detail_content = []
    
    # Question information
    detail_content.append(html.H4(f"Question ID: {question_id}"))
    
    # Section and category info
    section_name = {
        1: "Section 1 (Non-Verbal Reasoning)",
        2: "Section 2 (Logical Reasoning)",
        3: "Section 3 (Business Understanding)"
    }.get(question["section_id"], f"Section {question['section_id']}")
    
    detail_content.append(
        dbc.Row([
            dbc.Col([
                html.P(f"Section: {section_name}"),
            ], width=6),
            dbc.Col([
                html.P(f"Category: {question['category_name']}"),
            ], width=6),
        ])
    )
    
    detail_content.append(
        dbc.Row([
            dbc.Col([
                html.P(f"Question Type: {question['question_type'].replace('_', ' ').title()}"),
            ], width=6),
            dbc.Col([
                html.P(f"Difficulty: {question['difficulty']}/10"),
            ], width=6),
        ])
    )
    
    # Question text
    if question.get("question_text"):
        detail_content.append(html.Hr())
        detail_content.append(html.H5("Question Text:"))
        detail_content.append(html.P(question["question_text"]))
    
    # Question image
    if question.get("image_path"):
        detail_content.append(html.Hr())
        detail_content.append(html.H5("Question Image:"))
        
        # IMPORTANT CHANGE: Direct image serving implementation
        # Create a data URL by reading the image file and encoding it
        try:
            image_path = os.path.join(IMAGE_UPLOAD_DIRECTORY, question['image_path'])
            if os.path.exists(image_path):
                # Read the image file
                with open(image_path, 'rb') as img_file:
                    encoded_image = base64.b64encode(img_file.read()).decode('ascii')
                
                # Get file extension to determine MIME type
                file_ext = os.path.splitext(question['image_path'])[1].lower()
                mime_type = 'image/jpeg' if file_ext in ['.jpg', '.jpeg'] else 'image/png'
                
                # Create a data URL
                img_src = f'data:{mime_type};base64,{encoded_image}'
                detail_content.append(html.Img(src=img_src, style={"max-width": "100%"}))
            else:
                detail_content.append(html.P("Image file not found.", className="text-danger"))
        except Exception as e:
            detail_content.append(html.P(f"Error loading image: {str(e)}", className="text-danger"))
        
        detail_content.append(html.P(f"Image filename: {question['image_path']}"))
    
    # Answer options
    if answers:
        detail_content.append(html.Hr())
        detail_content.append(html.H5("Answer Options:"))
        
        # Create a more visual representation of answers
        options_list = []
        for i, answer in enumerate(answers):
            option_label = chr(65 + i)  # A, B, C, etc.
            
            # Style correct answers in green
            if answer["is_correct"]:
                option_style = {"background-color": "#d4edda", "border-radius": "5px", "padding": "5px", "margin-bottom": "5px"}
                correct_badge = html.Span("Correct", className="badge bg-success ms-2")
                option_text = [f"Option {option_label}: {answer['option_text']}", correct_badge]
            else:
                option_style = {"background-color": "#f8f9fa", "border-radius": "5px", "padding": "5px", "margin-bottom": "5px"}
                option_text = f"Option {option_label}: {answer['option_text']}"
            
            options_list.append(html.Div(option_text, style=option_style))
        
        detail_content.append(html.Div(options_list))
    
    return True, detail_content

# Callback to store question ID for deletion and toggle confirmation modal
@app.callback(
    [Output("delete-confirmation-modal", "is_open"),
     Output("question-to-delete", "data")],
    [Input({"type": "delete-question", "index": dash.ALL}, "n_clicks"),
     Input("confirm-delete", "n_clicks"),
     Input("cancel-delete", "n_clicks")],
    [State("delete-confirmation-modal", "is_open"),
     State("question-to-delete", "data")]
)
def toggle_delete_modal(delete_clicks, confirm_clicks, cancel_clicks, is_open, question_id):
    # Check which input triggered the callback
    triggered_id = ctx.triggered_id
    
    if triggered_id == "confirm-delete" and is_open:
        # Modal is open and user confirmed deletion
        return False, question_id
    
    if triggered_id == "cancel-delete":
        # User canceled deletion
        return False, None
    
    if isinstance(triggered_id, dict) and triggered_id.get("type") == "delete-question":
        # User clicked a delete button for a specific question
        if any(click for click in delete_clicks if click):
            return True, triggered_id["index"]
    
    # Default case - no change
    return is_open, question_id

# Callback to handle the actual deletion
@app.callback(
    Output("delete-feedback", "children"),
    [Input("question-to-delete", "data")],
    [State("delete-feedback", "children")]
)
def delete_question_callback(question_id, current_feedback):
    if question_id is None:
        return current_feedback
    
    # Perform the deletion
    success = delete_question(question_id)
    
    if success:
        return dbc.Alert(f"Question ID {question_id} deleted successfully", color="success", dismissable=True)
    else:
        return dbc.Alert(f"Failed to delete Question ID {question_id}", color="danger", dismissable=True)

# Callback to update filter-category-dropdown based on section selection
@app.callback(
    Output("filter-category-dropdown", "value"),
    [Input("filter-section-dropdown", "value")],
    [State("filter-category-dropdown", "options")]
)
def update_filter_category(section_id, category_options):
    # If "all" sections is selected, keep "all" categories
    if section_id == "all":
        return "all"
    
    # Map section IDs to default category names
    section_to_category = {
        "1": "Non-verbal Reasoning",
        "2": "Logical Reasoning",
        "3": "Business Understanding"
    }
    
    # If this section has a default category, find its ID
    if section_id in section_to_category:
        default_category_name = section_to_category[section_id]
        for option in category_options:
            if option["label"] == default_category_name:
                return option["value"]
    
    # If no matching category found, keep "all" categories
    return "all"

# Function to serve uploaded images (if needed)
# Note: A better approach would be to configure your web server (e.g., nginx) to serve these files directly
@app.server.route('/question_images/<path:filename>')
def serve_image(filename):
    return send_from_directory(IMAGE_UPLOAD_DIRECTORY, filename)

# Run the app
if __name__ == "__main__":
    app.run_server(debug=True, port=8085)

In [4]:
pip install dash_daq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 676.0/676.0 kB 669.1 kB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.
